In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import operator
import gc
import time

import nltk
stopwords = nltk.corpus.stopwords.words("english")
import string
import re

import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [2]:
train = pd.read_csv('./data/train_tweets.csv')
dev = pd.read_csv('./data/dev_tweets.csv')
test = pd.read_csv('./data/test_tweets.csv')

train['tweet'] = train['tweet'].astype(str)
dev['tweet'] = dev['tweet'].astype(str)
test['tweet'] = test['tweet'].astype(str)

In [3]:
print('Train shape:', train.shape)
print('Dev shape  :', dev.shape)
print('Test shape :', test.shape)

Train shape: (96585, 4)
Dev shape  : (34028, 4)
Test shape : (32977, 4)


In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for special in specials:
        text = text.replace(special, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(' ')])
    return text

In [5]:
# remove '@USER_*****'
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))

# clean contruction
train['tweet'] = train['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
dev['tweet'] = dev['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['tweet'] = test['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))

# lower text
train['tweet'] = train['tweet'].apply(lambda x: x.lower())
dev['tweet'] = dev['tweet'].apply(lambda x: x.lower())
test['tweet'] = test['tweet'].apply(lambda x: x.lower())

# remove stop words
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))

# remove non alphabetic chars
train['tweet'] = train['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
dev['tweet'] = dev['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
test['tweet'] = test['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))

# strip text
train['tweet'] = train['tweet'].apply(lambda x: x.strip())
dev['tweet'] = dev['tweet'].apply(lambda x: x.strip())
test['tweet'] = test['tweet'].apply(lambda x: x.strip())

In [6]:
def merge_same_user(df):
    df_class = df[['user-id', 'class']].groupby('user-id').apply(lambda x: x['class'].mode()).reset_index()
    df_class = df_class.rename(columns={0: 'class'})
    df_new = df[['user-id', 'tweet']].groupby('user-id').agg(lambda x: ' '.join(x))
    df_new = pd.merge(df_new, df_class, on='user-id')
    
    return df_new

In [7]:
train_per_user = merge_same_user(train)
dev_per_user = merge_same_user(dev)

In [8]:
merged = pd.concat([train_per_user, dev_per_user], axis=0)

vectorizer = CountVectorizer(max_features=20000)
vectorizer.fit(merged['tweet'])

merged_vec = vectorizer.transform(merged['tweet'])
merged_vec = pd.DataFrame(merged_vec.todense(), columns=vectorizer.get_feature_names())

In [9]:
chi_selector = SelectKBest(chi2, k=500)
chi_selector.fit(merged_vec, merged['class'])
chi_columns = merged_vec.columns[chi_selector.get_support()]

mi_selector = SelectKBest(mutual_info_classif, k=500)
mi_selector.fit(merged_vec, merged['class'])
mi_columns = merged_vec.columns[mi_selector.get_support()]

all_columns = chi_columns
all_columns = all_columns.append(mi_columns)
all_columns = np.array(all_columns, dtype=str)
all_columns = np.unique(all_columns)
np.save('best500', all_columns)

In [10]:
len(all_columns)

975